In [1]:
import pandas as pd
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Dense, Activation, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import SGD

In [2]:
import spacy
from nltk.tokenize import word_tokenize
# loading english language model of spaCy
en_model = spacy.load('en_core_web_sm')
# gettign the list of default stop words in spaCy english model
stopwords = en_model.Defaults.stop_words

In [3]:
#Add custom stopwords
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
df = pd.read_csv(r'C:\Users\joshu\Downloads\WikiQACorpus\WikiQACorpus\WikiQA.tsv', sep='\t')

In [6]:
df.columns

Index(['QuestionID', 'Question', 'DocumentID', 'DocumentTitle', 'SentenceID',
       'Sentence', 'Label'],
      dtype='object')

In [4]:
df = pd.read_csv(r"C:\Users\joshu\Downloads\twcs.csv\twcs.csv")
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


# Thought 2
Lets look at AppleSupport AmazonHelp, SpotifyCares TMobileHelp comcastcares

# Thought 1
Chatbot analyzes whether the text input is a question ?

If question, lookup answer

If not question, get sassy (humor)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [5]:
a_df = df[df['author_id'].isin(['AppleSupport', 'AmazonHelp', 'SpotifyCares','TMobileHelp', 'comcastcares'])]

In [6]:
#This is the questions being asked to the companys (our chatbot)
#TODO add the rest of the Company Names to the list comprehension
comp_df = df[df['in_response_to_tweet_id'].isna() & (df['text'].str.contains('@sprintcare') | df['text'].str.contains('@AppleSupport') | df['text'].str.contains('@AmazonHelp')) ]

In [ ]:
comp_df.to_csv('comp_df.csv')

# Intent Categories
1. Marketing
2. Commerce
3. Software
4. Tableau
5. Employees
6. Contact Management

# Thoughts 3
k means clustering (7) to the data.
Assign an intent based on the results

In [7]:
#id / tweet
df_id_tweet = comp_df[['tweet_id', 'text']]

Thoughts 4

TODO need to select a Color scheme for the graphs

In [8]:
df_id_tweet['tokens'] = df_id_tweet['text'].apply(lambda x: word_tokenize(x))

C:\Users\joshu\AppData\Local\Temp/ipykernel_18852/1944919945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_tweet['tokens'] = df_id_tweet['text'].apply(lambda x: word_tokenize(x))


In [9]:
#Add custom stopwords 'AppleSupport', 'AmazonHelp', 'SpotifyCares','TMobileHelp', 'comcastcares'
swords = ['AppleSupport', 'AmazonHelp','SpotifyCares','comcastcares','TMobileHelp','sprintcare']
for word in swords:
    stopwords.add(word)

In [10]:
def removeStopWords(token):
    '''Return only tokens not in stopwords'''
    results = [word for word in token if not word in stopwords]
    results = [word for word in results if not word in string.punctuation]    
    return results

In [11]:
df_id_tweet['cleanedTokens'] = df_id_tweet['tokens'].apply(removeStopWords)

C:\Users\joshu\AppData\Local\Temp/ipykernel_18852/2869727416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_tweet['cleanedTokens'] = df_id_tweet['tokens'].apply(removeStopWords)


In [12]:
freq_count = dict()
for tweet in df_id_tweet['cleanedTokens']:
    for word in tweet:
        if len(word) < 2:
            continue
        elif word not in freq_count:
            freq_count[word] = 1
        elif word in freq_count:
            freq_count[word] +=1


Get top 20 Frequency Counts of words

In [13]:
import matplotlib.pyplot as plt
lists = pd.DataFrame(sorted(freq_count.items()), columns=['name','count'])

In [18]:
df_id_tweet.head()

,tweet_id,text,tokens,cleanedTokens
6,8,@sprintcare is the worst customer service,"[@, sprintcare, is, the, worst, customer, serv...","[worst, customer, service]"
340,634,@115821 @AmazonHelp why is my order at my loca...,"[@, 115821, @, AmazonHelp, why, is, my, order,...","[115821, order, local, courier, 6, days, hasn,..."
349,643,Bought an @115821 Echo Show and it won’t recog...,"[Bought, an, @, 115821, Echo, Show, and, it, w...","[Bought, 115821, Echo, Show, won, ’, t, recogn..."
353,646,.@AmazonHelp Item has not been delivered but t...,"[., @, AmazonHelp, Item, has, not, been, deliv...","[Item, delivered, tracking, says, handed, hour..."
361,652,@AmazonHelp Is it possible to prevent AMZL fro...,"[@, AmazonHelp, Is, it, possible, to, prevent,...","[Is, possible, prevent, AMZL, delivering, pack..."


In [14]:
lists[lists['name'].isin(['employee','sales','marketing'])]

,name,count
49377,employee,39
57632,marketing,14
65393,sales,45


In [16]:
lists.sort_values(by='count', ascending=False).head(50)

,name,count
53615,https,16652
61343,phone,10993
53932,iPhone,9640
18374,115858,7878
31269,I️,7733
70596,update,7417
53809,iOS,6738
53134,help,5626
50940,fix,5434
59148,new,4844


In [63]:
df_id_tweet['Marketing'] = 0
df_id_tweet['Service'] = 0
df_id_tweet['Employees'] = 0
df_id_tweet['ContactManagement'] = 0
df_id_tweet['Commerce'] = 0
df_id_tweet['Mulesoft'] = 0
df_id_tweet['Tableau'] = 0

In [66]:
#Apply Labels to each of the tweets 
df_id_tweet.loc[df_id_tweet['text'].str.contains('marketing'),'Marketing'] = 1
service = ['service','english','commerce','digital','sales','115858', 'delivery']
df_id_tweet.loc[df_id_tweet['text'].str.contains('|'.join(service)), 'Service'] = 1
df_id_tweet.loc[df_id_tweet['text'].str.contains('employee'), 'Employees'] = 1
tableau = ['server','community','toggle','tableau','management']
df_id_tweet.loc[df_id_tweet['text'].str.contains('|'.join(tableau)), 'Tableau'] = 1
contactMgmnt = ['contact','updated']
df_id_tweet.loc[df_id_tweet['text'].str.contains('|'.join(contactMgmnt)), 'ContactManagement'] = 1

In [67]:
#Count the number of each intent
print(f"Tableau Count: {df_id_tweet[df_id_tweet['Tableau']== 1]['Tableau'].count()}")
print(f"Marketing Count: {df_id_tweet[df_id_tweet['Marketing']== 1]['Marketing'].count()}")
print(f"Service Count: {df_id_tweet[df_id_tweet['Service']== 1]['Tableau'].count()}")
print(f"Employees Count: {df_id_tweet[df_id_tweet['Employees']== 1]['Tableau'].count()}")
print(f"Contact Count: {df_id_tweet[df_id_tweet['ContactManagement']== 1]['Tableau'].count()}")

Tableau Count: 244
Marketing Count: 16
Service Count: 13148
Employees Count: 72
Contact Count: 3290


# Split Train data

In [72]:
X = df_id_tweet['cleanedTokens']
y = df_id_tweet[['Tableau','Marketing','Service','Employees','ContactManagement']]

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [80]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)

ValueError: Expected 2D array, got 1D array instead:
array=[list(['worst', 'customer', 'service'])
 list(['115821', 'order', 'local', 'courier', '6', 'days', 'hasn', '’', 't', 'delivered', 'Over', '1', 'week', 'late', '😡'])
 list(['Bought', '115821', 'Echo', 'Show', 'won', '’', 't', 'recognize', 'single', 'account', 'household', 'WTF', 'guys'])
 ...
 list(['Yo', 'weird', 'glitch', 'w/', 'capital', '“', 'I️', '”', 'attempt', 'tweets', 'I️'])
 list(['What', 'fuck', 'phone', 'keeps', 'hanging', 'calls', 'showing', '“', 'failure', '”', '🤬😡'])
 list(['Is', 'having', 'iphone', 'issues', 'phone', 'freezing', 'randomly', 'I', '7', 'pulse', 'updated', 'iOS', '11.2', 'newest', 'version', 'Wtf', 'going', '🤬😡🤬😡🤬', '115858', 'whycan', '’', 'tyouguysgetitright', 'stopmakingnewphonesuntillyoulearntomaketheoldonerunrite'])].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [75]:
y_train[0:5]

,Tableau,Marketing,Service,Employees,ContactManagement
695978,0,0,0,0,0
655492,0,0,0,0,0
1101015,0,0,0,0,0
592898,0,0,1,0,0
1024112,0,0,0,0,0


In [11]:
#Cleanup data
df['in_response_to_tweet_id'].dropna().astype(int)

0                3
1                1
2                4
3                5
4                6
            ...   
2811766     139627
2811767    2987946
2811769    2987948
2811771    2812239
2811772    2987950
Name: in_response_to_tweet_id, Length: 2017439, dtype: int32

In [10]:
#Import SpaCy language
from spacy.lang.en import English
nlp = English()

In [26]:
authors = list(df['author_id'].unique())

In [28]:
#replies
replies = df[df['text'].str.contains('@sprintcare')]

In [31]:
replies.text = replies.text.astype(str)


C:\Users\joshu\anaconda3\envs\chatbot\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
a_df = df[df['author_id'].isin(['AppleSupport', 'AmazonHelp','Uber_suport','sprintcare', 'VerizonSupport'])]